# Shor's factoring algorithm

In [1]:
from qsystem import QSystem, Gates
from random import random, randint
from math import log2, ceil, floor, gcd
from IPython.display import display, Latex, Math

In [2]:
N = 15
display(Latex('Factoring $N = {}$'.format(N)))

<IPython.core.display.Latex object>

## step 1: Random select an $a$ less than $N$ and coprime with $N$

In [3]:
a = 0
while gcd(N, a) != 1 or a == 1:
    a = floor(random() * 10000) % N
display(Latex('$a = {}$'.format(a)))

<IPython.core.display.Latex object>

## step 2: use the quantum period-finding to find the period $r$ of the function $f(x) = a^x(\text{mod}\,n)$

### Period-finding

It will be necessary 2 quantum registers with size of 
$$n = \lceil\log_2(N+1)\rceil$$
and a quantum oracle that 
$$\left|x\right>\left|0\right>
\xrightarrow{\text{POWN}}
\left|x\right>\left|a^x (\text{mod}\,N)\right>$$

In [4]:
n = ceil(log2(N+1))
display(Latex('$n = {}$'.format(n)))

def POWN(n, a, N):
    row, col, value = [], [], []
    for x in range(2**n):
        fx = pow(a, x, N)
        row.append((x << n) | fx)
        col.append(x << n)
        value.append(1)
    return row, col, value
gates = Gates()        

row, col, value = POWN(n, a, N)
gates.make_mgate('POWN', n*2, row, col, value)

<IPython.core.display.Latex object>

### step 1: Prepare a superposition
$$\left|0\right>\left|0\right>
\xrightarrow{H^{\otimes n}}
{1\over\sqrt{2^n}}\sum_{x=0}^{2^{n}-1} \left|x\right>\left|0\right>
$$

In [5]:
seed = randint(0,10000)
q = QSystem(n, gates, seed)
q.add_ancillas(n)

q.evol(gate='H', qbit=0, count=n)
print(q)

+0.250       |0000>|0000>
+0.250       |0001>|0000>
+0.250       |0010>|0000>
+0.250       |0011>|0000>
+0.250       |0100>|0000>
+0.250       |0101>|0000>
+0.250       |0110>|0000>
+0.250       |0111>|0000>
+0.250       |1000>|0000>
+0.250       |1001>|0000>
+0.250       |1010>|0000>
+0.250       |1011>|0000>
+0.250       |1100>|0000>
+0.250       |1101>|0000>
+0.250       |1110>|0000>
+0.250       |1111>|0000>



### step 2: Prepare a periodic superposition
$$
{1\over\sqrt{2^n}}\sum_{x=0}^{2^{n}-1} \left|x\right>\left|0\right>
\xrightarrow{\text{POWN}}
{1\over\sqrt{2^n}}\sum_{x=0}^{2^{n}-1}
\left|x\right>\left|a^x(\text{mod}\, N)\right>
$$

In [6]:
q.evol(gate='POWN', qbit=0)
print(q)

+0.250       |0000>|0001>
+0.250       |0001>|0111>
+0.250       |0010>|0100>
+0.250       |0011>|1101>
+0.250       |0100>|0001>
+0.250       |0101>|0111>
+0.250       |0110>|0100>
+0.250       |0111>|1101>
+0.250       |1000>|0001>
+0.250       |1001>|0111>
+0.250       |1010>|0100>
+0.250       |1011>|1101>
+0.250       |1100>|0001>
+0.250       |1101>|0111>
+0.250       |1110>|0100>
+0.250       |1111>|1101>



### step 3 (optional): measure the second register
help to understand the algorithm

$$
{1\over\sqrt{2^n}}\sum_{x=0}^{2^{n}-1}
\left|x\right>\left|a^x(\text{mod}\, N)\right>
\xrightarrow{\text{measure}[n:2n]}
\sqrt{r\over{2^n}}\sum_{i=0}^{{2^{n}\over r}-1}
\left|ir + x_0\right>\left|a^{x_0}(\text{mod}\, N)\right>
$$


In [7]:
q.measure(qbit=n, count=n)
print(q)

+0.500       |0001>|0111>
+0.500       |0101>|0111>
+0.500       |1001>|0111>
+0.500       |1101>|0111>



### step 4: fourier transform of the first register
$$
\sqrt{r\over{2^n}}\sum_{i=0}^{{2^{n}\over r}-1}
\left|ir + x_0\right>
\xrightarrow{\text{QFT}_n}
{1\over\sqrt{r}}\sum_{i=0}^{r-1}\left|i{2^n\over r}\right>\phi_i
$$

In [8]:
q.qft(qbegin=0, qend=n)
print(q)

+0.500       |0000>|0111>
      +0.500i|0100>|0111>
-0.500       |1000>|0111>
      -0.500i|1100>|0111>



### step 5: measure the first register and repeat the algorithm to measure distincts multiples of $2^n\over r$

In [9]:
q.measure(0, n)
c = q.bits()[0:n]
c = sum([m*2**i for m, i in zip(c, reversed(range(len(c))))])
mea = [c]

for _ in range(n-1):
    seed = randint(0,10000)
    q = QSystem(n, gates, seed)
    q.add_ancillas(n)

    q.evol('H', 0, n) # 1
    q.evol('POWN', 0) # 2    
    q.qft(0, n)       # 4
    q.measure(0, n)   # 5

    c = q.bits()[0:n]
    c = sum([m*2**i for m, i in zip(c, reversed(range(len(c))))])
    mea.append(c)
print('measurements results =', mea)

measurements results = [0, 8, 0, 4]


### step 6: with the measurements compute 
$$
r = {2^n\over\gcd(\text{measurements})}
$$

In [10]:
c = mea[0]
for m in mea:
    c = gcd(c, m)
if c == 0:
    print('repite the period-finding algorithm')
else:
    r = 2**n/c
    display(Latex('possible period $r = {}$'.format(r)))

<IPython.core.display.Latex object>

## step 3: if $r$ is odd go to step 1 else compute the two nontrivial factors of $N$, $pq = N$
$$
p = \gcd(a^{r\over2}-1, N)
$$
$$
q = \gcd(a^{r\over2}+1, N)
$$

In [11]:
if c == 0:
    print('repite the period-finding algorithm')
elif r % 2 == 1:
    print('go to step 1')
else: 
    p = gcd(int(a**(r/2)+1), N)
    q = gcd(int(a**(r/2)-1), N)
    display(Math('{}*{}={}'.format(p,q,p*q)))

<IPython.core.display.Math object>